In [1]:
from utils.Data import Data
from utils.Camera import Camera

In [2]:
data = Data()
camera = Camera('data/camera.dat')

INFO - Loading trajectory data from data//trajectory.dat
INFO - 0.00 [s] - Trajectory data loaded successfully!
INFO - Loading world data from data//world.dat
INFO - 0.00 [s] - World data loaded successfully!
INFO - Loading measurements data from data//meas-*.dat
INFO - 0.06 [s] - Measurements data loaded successfully!
INFO - Loading camera parameters from data/camera.dat
INFO - 0.00s - Camera parameters loaded successfully.


In [3]:
def compute_points(measurement):
    gt_pose_1 = measurement['gt_pose']
    odom_pose_1 = measurement['odom_pose']
    points_1 = {'point_id':[], 'image_point':[], 'camera_point':[], 'world_point':[], 'appearance':[]}  

    for point in measurement['points']:    

        points_1['point_id'].append(measurement['points'][point]['actual_point_id'])

        image_point = measurement['points'][point]['image_point']
        points_1['image_point'].append(image_point)

        camera_point = camera.pixel_to_camera(image_point)
        points_1['camera_point'].append(camera_point)

        world_point = camera.camera_to_world(camera_point)
        points_1['world_point'].append(world_point)

        points_1['appearance'].append(measurement['points'][point]['appearance'])

    return gt_pose_1, odom_pose_1, points_1

In [4]:
def data_association(points_1, points_2):
    matches = dict()
    for i in range(len(points_1['point_id'])):
        for j in range(len(points_2['point_id'])):
            if points_1['appearance'][i] == points_2['appearance'][j]:
                id = points_1['point_id'][i]
                world_point_1 = points_1['world_point'][i]
                world_point_2 = points_2['world_point'][j]
                matches[id] = [world_point_1, world_point_2]
    return matches

In [5]:
import numpy as np
def ICP(set1, set2):

    '''
    Given two sets of points, set1 and set2, which are the corresponding points in two different coordinate systems,
    this function returns the rotation and translation matrix that best aligns the two sets of points.
    '''

    # Calculate the centroid of each set of points
    x_0 = np.mean(set1, axis=0)
    y_0 = np.mean(set2, axis=0)

    # Subtract the centroid from each set of points
    set1 = set1 - x_0
    set2 = set2 - y_0

    # Calculate the covariance matrix of the two sets of points
    H = np.dot(set1.T, set2)
    
    # Use the SVD to calculate the rotation matrix
    U, S, Vt = np.linalg.svd(H)

    # Calculate the rotation matrix
    R = np.dot(Vt.T, U.T)

    # Calculate the translation matrix
    T = y_0 - np.dot(R, x_0)

    return R, T

In [6]:
R = np.eye(3)
T = np.zeros(3)
estimated_trajectory = []

for i in range(120):
    measurement_1 = data.get_measurements(i)
    measurement_2 = data.get_measurements(i+1)

    gt_pose_1, odom_pose_1, points_1 = compute_points(measurement_1)
    gt_pose_2, odom_pose_2, points_2 = compute_points(measurement_2)

    matches = data_association(points_1, points_2)

    set1 = np.array([matches[key][0] for key in matches])
    set2 = np.array([matches[key][1] for key in matches])

    estimated_R, estimated_T = ICP(set1, set2)

    R = np.dot(R, estimated_R)
    T = T + np.dot(R, estimated_T)

    if i == 1:
        print(points_1['point_id'])
        print(points_1['world_point'])


    updated_pose = np.dot(estimated_R, odom_pose_1) + estimated_T

    estimated_trajectory.append(updated_pose)

[6, 14, 15, 17, 35, 36, 38, 42, 44, 45, 53, 54, 56, 57, 59, 61, 64, 69, 84, 89, 106, 110, 113, 115, 119, 120, 130, 132, 134, 140, 148, 152, 157, 158, 159, 161, 165, 169, 170, 179, 198, 202, 259, 261, 280, 286, 290, 296, 301, 310, 313, 315, 316, 319, 325, 327, 343, 353, 367, 374, 396, 398, 411, 444, 446, 448, 449, 452, 463, 468, 469, 481, 482, 487, 516, 519, 532, 540, 571, 573, 581, 588, 603, 629, 639, 646, 655, 658, 664, 667, 673, 681, 697, 709, 717, 731, 743, 771, 777, 791, 804, 815, 821, 837, 846, 867, 868, 878, 882, 885, 886, 903, 905, 910, 918, 920, 927, 928, 935, 939, 963, 971, 979, 989]
[array([1.2       , 1.77397561, 1.33202829]), array([1.2       , 1.77528318, 1.33070484]), array([1.2       , 1.78234957, 1.33177711]), array([1.2       , 1.77803248, 1.33033671]), array([1.2       , 1.7743341 , 1.33275963]), array([1.2       , 1.78230825, 1.33317657]), array([1.2       , 1.77339872, 1.33092609]), array([1.2       , 1.77809134, 1.33180104]), array([1.2       , 1.78029547, 1.331418

In [7]:
data.print_world(5)

World data: 
landmark_id: 5
  landmark_position:   [0.795207, -2.49586, 1.04099]
  landmark_appearance: [0.0250708, 0.335448, 0.0632129, -0.921439, -0.124725, 0.86367, 0.86162, 0.441905, -0.431413, 0.477069]


In [8]:
trajectory_data = data.get_trajectory()
gt_trajectory = []
odom_trajectory = []
for key, value in trajectory_data.items():
    gt_trajectory.append(value['ground_truth_pose'])
    odom_trajectory.append(value['odometry_pose'])

In [9]:
print(gt_trajectory)

[[0.0, 0.0, 0.0], [0.200426, 0.0, 0.0], [0.40085, 0.0, 0.0], [0.601274, 0.0, 0.0], [0.801698, 0.0, 0.0], [1.00244, 0.0, 0.0], [1.20254, 0.0, 0.0], [1.40263, 0.0, 0.0], [1.60272, 0.0, 0.0], [1.80282, 0.0, 0.0], [2.00291, 0.0, 0.0], [2.20353, 0.0, 0.0], [2.40355, 0.0, 0.0], [2.60358, 0.0, 0.0], [2.8036, 0.0, 0.0], [3.00362, 0.0, 0.0], [3.20364, 0.0, 0.0], [3.40319, 0.0151316, 0.188353], [3.56783, 0.0638916, 0.388423], [3.71949, 0.144398, 0.58849], [3.85213, 0.253439, 0.788559], [3.96045, 0.386664, 0.988628], [4.04013, 0.538759, 1.1887], [4.08799, 0.703654, 1.38877], [4.1131, 0.902461, 1.45456], [4.13633, 1.10141, 1.45456], [4.15956, 1.30036, 1.45456], [4.18278, 1.4993, 1.45456], [4.20601, 1.69824, 1.45456], [4.22923, 1.89719, 1.45456], [4.25246, 2.09613, 1.45456], [4.27568, 2.29488, 1.45456], [4.29899, 2.49425, 1.45456], [4.32229, 2.69361, 1.45456], [4.34559, 2.89298, 1.45456], [4.36889, 3.09234, 1.45456], [4.39219, 3.29171, 1.45456], [4.41549, 3.49107, 1.45456], [4.4388, 3.69044, 1.4545

In [10]:
print(odom_trajectory)

[[0.00160159, 0.0, -0.000259093], [0.200147, 0.00177571, 0.018434], [0.404335, 0.00659056, 0.0230666], [0.612285, 0.00727619, -0.00321606], [0.798097, 0.00376101, -0.0356316], [0.996043, -0.00186033, -0.0264914], [1.19474, -0.00818324, -0.0297342], [1.38451, -0.0135787, -0.0253994], [1.58873, -0.0196275, -0.0321115], [1.79787, -0.0231446, -0.0108615], [2.004, -0.0276231, -0.0309834], [2.21655, -0.0351534, -0.0426189], [2.41501, -0.0414293, -0.0211719], [2.6353, -0.0456665, -0.0101662], [2.83595, -0.0480199, -0.00676776], [3.00959, -0.0489147, 0.00463268], [3.20072, -0.0493376, -0.0106586], [3.39648, -0.0381659, 0.154862], [3.5805, 0.0111893, 0.359332], [3.73382, 0.0865316, 0.550893], [3.87529, 0.192391, 0.744261], [3.99312, 0.325085, 0.94226], [4.0863, 0.48363, 1.1358], [4.13753, 0.634739, 1.34335], [4.17609, 0.840069, 1.37262], [4.2157, 1.04307, 1.37542], [4.2486, 1.20768, 1.36706], [4.28921, 1.39129, 1.34276], [4.33069, 1.56984, 1.34083], [4.37772, 1.77115, 1.34605], [4.42459, 1.9869

In [11]:
print(estimated_trajectory)

[array([ 0.00160159, -0.00173889,  0.00197637]), array([ 2.00147000e-01, -1.56064265e-04,  2.09559333e-02]), array([0.404335  , 0.00339343, 0.02726162]), array([ 6.12285000e-01,  5.18534819e-03, -6.06246477e-04]), array([ 0.798097  ,  0.00160017, -0.03295189]), array([ 0.996043  , -0.00378418, -0.02412307]), array([ 1.19474   , -0.00822431, -0.02986102]), array([ 1.38451   , -0.01216228, -0.02738347]), array([ 1.58873   , -0.01590207, -0.03711719]), array([ 1.79787   , -0.01987361, -0.01538515]), array([ 2.004     , -0.02770232, -0.03100916]), array([ 2.21655   , -0.03468594, -0.04338695]), array([ 2.41501   , -0.04217922, -0.02025852]), array([ 2.6353    , -0.048998  , -0.00568712]), array([ 2.83595   , -0.04940396, -0.00495829]), array([ 3.00959   , -0.05046544,  0.00663582]), array([ 3.20072   ,  0.04648387, -0.13346685]), array([3.39648   , 0.04601589, 0.03271203]), array([3.5805    , 0.08627111, 0.23230629]), array([3.73382   , 0.14041733, 0.44158651]), array([3.87529   , 0.235454

In [12]:
max_points = 100
estimated_trajectory_to_plot = estimated_trajectory[:max_points]
gt_trajectory_to_plot = gt_trajectory[:max_points]
odom_trajectory_to_plot = odom_trajectory[:max_points]

In [13]:
# plot the estimated_trajectory in blue, the ground truth in green, and the odometry in red with with a widget plot that can be rotated interactively with the mouse

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter3d(x=[pose[0] for pose in estimated_trajectory_to_plot], y=[pose[1] for pose in estimated_trajectory_to_plot], z=[pose[2] for pose in estimated_trajectory_to_plot], mode='lines', name='Estimated trajectory', line=dict(color='blue', width=5)))
fig.add_trace(go.Scatter3d(x=[pose[0] for pose in gt_trajectory_to_plot], y=[pose[1] for pose in gt_trajectory_to_plot], z=[pose[2] for pose in gt_trajectory_to_plot], mode='lines', name='Ground truth trajectory', line=dict(color='green', width=5)))
fig.add_trace(go.Scatter3d(x=[pose[0] for pose in odom_trajectory_to_plot], y=[pose[1] for pose in odom_trajectory_to_plot], z=[pose[2] for pose in odom_trajectory_to_plot], mode='lines', name='Odometry trajectory', line=dict(color='red', width=5)))

fig.show()